# Processing larger than RAM arrays

In [20]:
import sys

import gzip
import pickle
import torch
import numpy as np
import pandas as pd

In [33]:
def load_gzipped_pytorch_embeddings(filename):
    """Load compressed embeddings
    """
    
    with gzip.open(filename, 'rb') as emb_f:
        embeddings = pickle.load(emb_f)
    # file = gzip.GzipFile(filename, 'rb')
    # data = file.read()
    # obj = pickle.loads(data)
    # file.close()
    return embeddings


In [ ]:
# big_embeddings_path = "../../../s3-buckets/cpr-datasets/policy_text_embeddings_multi-qa-MiniLM-L6-cos-v1.pkl.gz"
big_embeddings_path = "../../../s3-buckets/cpr-datasets/policy_text_embeddings_msmarco-distilbert-dot-v5.pkl.gz"
mappings_path = "../../../s3-buckets/cpr-datasets/policy_text_embeddings_mapping_multi-qa-MiniLM-L6-cos-v1.pkl.gz"

emb = load_gzipped_pytorch_embeddings(big_embeddings_path)

mapping = pd.read_pickle(mappings_path, compression="gzip")

In [ ]:
emb.dtype

In [ ]:
f"{sys.getsizeof(emb.storage()) / 1e9} GB"

In [ ]:
emb.numpy().shape

In [ ]:
emb_mm = np.memmap('../../../s3-buckets/cpr-datasets/policy_text_embeddings_msmarco-distilbert-dot-v5_dim_384.mymemmap', dtype='float32', mode='w+', shape=emb.numpy().shape)

In [ ]:
emb_mm[:] = emb.numpy()[:]

In [41]:
del emb
del emb_mm

In [42]:
mm_loaded = np.memmap('test.mymemmap', dtype='float32', mode='r+', shape=(1666918, 384))
mm_loaded.shape

(1666918, 384)

In [43]:
text_ids = mapping.loc[mapping['policy_id'] == 0, 'text_id']
mm_loaded[text_ids, :]

array([[-0.04356783,  0.07272697,  0.05683589, ...,  0.00689421,
        -0.07691474,  0.02155772],
       [ 0.03467601,  0.00806095,  0.04000822, ...,  0.04712495,
         0.05529409, -0.05920185],
       [-0.00875157,  0.09728127,  0.00627775, ..., -0.01215494,
        -0.0151676 ,  0.00647304],
       ...,
       [-0.00618983, -0.00748422, -0.02027573, ...,  0.04320186,
         0.01393319,  0.01562135],
       [ 0.04180384,  0.12031212,  0.00712933, ...,  0.06030804,
         0.01011585,  0.03136686],
       [ 0.0486473 ,  0.01465825, -0.04043028, ..., -0.02719378,
         0.073947  , -0.02428901]], dtype=float32)